# 🌱 ESG Stock Event Study — Mean-Adjusted Model (Demo)

This notebook loads the **daily-updated datasets** produced by the GitHub Actions pipeline and computes an event study using a **mean-adjusted model**:

- **Estimation window:** τ ∈ [-120, -20] days before the event  
- **Event window:** τ ∈ [-5, +5] days around the event  
- **Expected return:** mean return of the stock over the estimation window  
- **AR:** abnormal return = actual return − expected return  
- **AAR:** average abnormal return across events for each τ  
- **CAR:** cumulative abnormal return across τ


In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Try to locate the 'data/latest' directory whether you run this from
# the repo root or inside the project folder.
CANDIDATES = [
    "data/latest",
    "data-science-projects-starter/data-science-projects/esg-stock-event-study/data/latest"
]

DATA_LATEST = None
for p in CANDIDATES:
    if os.path.exists(p):
        DATA_LATEST = p
        break

if DATA_LATEST is None:
    raise FileNotFoundError("Could not find data/latest. Make sure you ran the pipeline or adjusted the path.")

prices_path   = os.path.join(DATA_LATEST, "prices_latest.csv")
returns_path  = os.path.join(DATA_LATEST, "returns_latest.csv")
events_path   = os.path.join(DATA_LATEST, "esg_events_latest.csv")
panel_path    = os.path.join(DATA_LATEST, "events_prices_panel.csv")

prices   = pd.read_csv(prices_path, parse_dates=[0], index_col=0)
returns  = pd.read_csv(returns_path, parse_dates=[0], index_col=0)
events   = pd.read_csv(events_path, parse_dates=['event_date']) if os.path.exists(events_path) else pd.DataFrame(columns=['ticker','event_date'])
panel    = pd.read_csv(panel_path, parse_dates=['date','event_date'])

prices.tail(), returns.tail(), events.head(), panel.head()

## 1) Prepare returns aligned to each event
We'll compute each event's **estimation window** ([-120, -20]) using the global `returns_latest.csv`,
then calculate **AR** within the event window ([-5, +5]).

In [ ]:
from datetime import timedelta

EST_WIN = (-120, -20)
EVT_WIN = (-5, 5)

def mean_adjusted_expected(ser, event_date, est_win=EST_WIN):
    start = (event_date + timedelta(days=est_win[0]))
    end   = (event_date + timedelta(days=est_win[1]))
    s = ser.loc[(ser.index.date >= start.date()) & (ser.index.date <= end.date())]
    if s.empty:
        return np.nan
    return s.mean()

def build_event_ar_table(returns_df, events_df, evt_win=EVT_WIN):
    rows = []
    r = returns_df.copy()
    r.index = pd.to_datetime(r.index)

    for _, ev in events_df.iterrows():
        tic = ev.get("ticker")
        if tic not in r.columns:
            continue
        event_date = pd.to_datetime(ev["event_date"]).normalize()

        mu = mean_adjusted_expected(r[tic], event_date, est_win=EST_WIN)
        if np.isnan(mu):
            continue

        start = event_date + timedelta(days=evt_win[0])
        end   = event_date + timedelta(days=evt_win[1])
        seg = r.loc[(r.index >= start) & (r.index <= end), [tic]].copy()
        if seg.empty:
            continue

        seg["date"] = seg.index
        seg["tau"] = (seg["date"].dt.normalize() - event_date).dt.days
        seg["ticker"] = tic
        seg["event_date"] = event_date
        seg.rename(columns={tic: "ret"}, inplace=True)
        seg["exp_ret"] = mu
        seg["ar"] = seg["ret"] - seg["exp_ret"]
        rows.append(seg[["date","ticker","event_date","tau","ret","exp_ret","ar"]])

    if not rows:
        return pd.DataFrame(columns=["date","ticker","event_date","tau","ret","exp_ret","ar"])
    out = pd.concat(rows, ignore_index=True)
    return out

event_ar = build_event_ar_table(returns, events)
event_ar.head(), event_ar["ticker"].nunique(), events.shape[0]

## 2) Compute AAR and CAR

In [ ]:
def aggregate_aar_car(event_ar_df):
    aar = event_ar_df.groupby("tau", as_index=False)["ar"].mean().sort_values("tau")
    aar.rename(columns={"ar":"AAR"}, inplace=True)

    car = aar.copy()
    car["CAR"] = car["AAR"].cumsum()
    return aar, car

aar, car = aggregate_aar_car(event_ar)
aar.head(), car.head()

## 3) Plot AAR and CAR

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.plot(aar["tau"], aar["AAR"], marker="o")
plt.axvline(0, linestyle="--")
plt.title("Average Abnormal Return (AAR)")
plt.xlabel("Tau (days relative to event)")
plt.ylabel("AAR")
plt.grid(True)
plt.show()

plt.figure()
plt.plot(car["tau"], car["CAR"], marker="o")
plt.axvline(0, linestyle="--")
plt.title("Cumulative Abnormal Return (CAR)")
plt.xlabel("Tau (days relative to event)")
plt.ylabel("CAR")
plt.grid(True)
plt.show()

## 4) Inspect one example event

In [ ]:
# Inspect a sample event window table
if not event_ar.empty:
    eg = event_ar.sort_values(["event_date","ticker","tau"]).groupby(["ticker","event_date"]).head(11)
    eg.head(15)
else:
    print("No event AR table generated. You may need more events or adjust tickers/keywords.")